In [1]:
from RobinRollandModel.datautils import TipGenerator, visualize
from RobinRollandModel.main import RRModel
import RobinRollandModel.pyironjob
import numpy as np
from pyiron import Project
import matplotlib.pyplot as plt

In [2]:
pr = Project("Example")

In [3]:
test = pr.create.job.RRModelAPTjob('test',delete_existing_job=True)

In [4]:
structure = pr.create.structure.bulk('Al','fcc',4.01,cubic=True)

In [5]:
visualize(structure)

In [6]:
tip = TipGenerator(structure,h=80,ah=20,zheight=50)

In [7]:
pr_tip = tip.create_tip_pyiron(pr)

In [8]:
visualize(pr_tip)

In [16]:
tip_shank = TipGenerator(structure=structure,h=50,ah=20,alpha=5,zheight=30)

In [17]:
pr_tip_shank = tip_shank.create_tip_pyiron(pr)

In [18]:
visualize(pr_tip_shank)

In [19]:
al_apt_sim = RRModel(tip_generator=tip,structure=pr_tip,e_field=4)

In [20]:
tip_output_al = al_apt_sim.charge_distribution_z(structure=pr_tip,e_field=4,radius=20,steps=1000,epsilon=1e-9)

/Users/shyamkatnagallu/Projects/APTrajectories/RobinRollandModel/main.py:62: RuntimeWarning:

invalid value encountered in divide



In [21]:
visualize(pr_tip,charge=tip_output_al['final_charge'],surf_indices=tip_output_al['surface_indices'])

In [ ]:
al_apt_sim.run_evaporation()

# pyiron implementation

In [11]:
pr = Project("Example_pyiron")

In [12]:
test = pr.create.job.RRModelAPTjob('test',delete_existing_job=True)

In [13]:
structure = pr.create.structure.bulk('Al','fcc',4.01,cubic=True)

In [14]:
test.input.basic_structure = structure
test.input.e_field = 4
test.input.tip_height = 80
test.input.tip_radius = 20
test.input.z_height = 50
test.input.num_atoms = 10

In [21]:
test.project?

Type:        property
String form: <property object at 0x2a05ef510>
Docstring:  
Project instance the jobs is located in

Returns:
    Project: project the job is located in

In [22]:
test.create

AttributeError: 'RRModelAPTjob' object has no attribute 'create'

In [16]:
test.run_static()

/Users/shyamkatnagallu/Projects/APTrajectories/RobinRollandModel/main.py:63: RuntimeWarning:

invalid value encountered in divide

/Users/shyamkatnagallu/Projects/APTrajectories/RobinRollandModel/main.py:158: RuntimeWarning:

invalid value encountered in scalar divide



ValueError: attempt to get argmax of an empty sequence

In [9]:
from cProfile import Profile
with Profile() as prof:
    tip = TipGenerator(structure,h=80,ah=20,zheight=50)
    pr_tip = tip.create_tip_pyiron(pr)
    al_apt_sim = RRModel(tip_generator=tip,structure=pr_tip,e_field=4)
    prof.dump_stats('my_apt_imp.pyprofile')
    prof.print_stats()

         378562 function calls (378556 primitive calls) in 0.629 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen genericpath>:121(_splitext)
        1    0.000    0.000    0.000    0.000 <frozen posixpath>:117(splitext)
        1    0.000    0.000    0.000    0.000 <frozen posixpath>:140(basename)
        1    0.000    0.000    0.000    0.000 <frozen posixpath>:41(_get_sep)
        3    0.000    0.000    0.000    0.000 <frozen posixpath>:52(normcase)
        1    0.000    0.000    0.000    0.000 __init__.py:123(getLevelName)
        1    0.000    0.000    0.000    0.000 __init__.py:1467(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1561(findCaller)
        1    0.000    0.000    0.000    0.000 __init__.py:1595(makeRecord)
        1    0.000    0.000    0.000    0.000 __init__.py:1610(_log)
        1    0.000    0.000    0.000    0.000 __init__.py:1636(

In [10]:
import pstats
stream = open('my_apt_imp.txt', 'w')
stats = pstats.Stats('my_apt_imp.pyprofile', stream=stream)
stats.sort_stats('tottime')
stats.print_stats()